# 第54章: カメラキャリブレーション

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] カメラキャリブレーションの目的と手法を理解する
- [ ] Zhang法（平面パターン法）を説明できる
- [ ] OpenCVでキャリブレーションを実行できる
- [ ] キャリブレーション精度の評価方法を理解する

## 🎯 前提知識

- ✅ Notebook 51-53（射影モデル、歪み、座標変換）

⏱️ **推定学習時間**: 150-180分  
📊 **難易度**: ★★★★☆（上級）  
🎓 **カテゴリ**: 理論 + 実践

---

## 🌟 はじめに

**カメラキャリブレーション**は、カメラの内部パラメータと歪み係数を推定するプロセスです。

```
キャリブレーションで推定するもの:

┌─────────────────────────────────────────────────────────┐
│  内部パラメータ K        │  歪み係数                     │
│  ┌─────────────────┐    │  [k1, k2, p1, p2, k3]        │
│  │ fx  0   cx │         │                               │
│  │ 0   fy  cy │         │  放射歪み: k1, k2, k3         │
│  │ 0   0   1  │         │  接線歪み: p1, p2             │
│  └─────────────────┘    │                               │
└─────────────────────────────────────────────────────────┘
```

**なぜキャリブレーションが必要か:**
- 正確な3D復元のため
- 歪みのない画像を得るため
- カメラ間の相対位置を知るため（ステレオ）

In [ ]:
# ============================================================
# 環境設定
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. キャリブレーションの目的

### 1.1 推定するパラメータ

| カテゴリ | パラメータ | 意味 |
|----------|-----------|------|
| **内部パラメータ** | fx, fy | 焦点距離（ピクセル単位）|
| | cx, cy | 主点（光軸と撮像面の交点）|
| **歪み係数** | k1, k2, k3 | 放射歪み |
| | p1, p2 | 接線歪み |
| **外部パラメータ** | R, t | 各画像でのカメラポーズ |

In [ ]:
# ============================================================
# キャリブレーションの概要図
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 左: チェッカーボードパターン
ax = axes[0]
checker = np.zeros((8, 8))
checker[1::2, ::2] = 1
checker[::2, 1::2] = 1
ax.imshow(checker, cmap='gray', interpolation='nearest')
ax.set_title('1. キャリブレーションターゲット\n（チェッカーボード）', fontsize=12, fontweight='bold')
ax.axis('off')

# 中央: 複数の角度から撮影
ax = axes[1]
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

# カメラアイコン
camera_positions = [
    (0, 1.5, 0),
    (-1.2, 1.2, -30),
    (1.2, 1.2, 30),
    (-0.8, 0.8, -20),
    (0.8, 0.8, 20),
]

for x, y, angle in camera_positions:
    ax.scatter(x, y, s=200, c='blue', marker='s')
    # 視線方向
    dx = -0.5 * np.sin(np.radians(angle))
    dy = -0.5 * np.cos(np.radians(angle))
    ax.arrow(x, y, dx, dy, head_width=0.1, head_length=0.05, fc='blue', ec='blue')

# チェッカーボード（中央）
ax.add_patch(plt.Rectangle((-0.3, -0.3), 0.6, 0.6, fill=True, facecolor='lightgray', edgecolor='black'))

ax.set_title('2. 複数の角度から撮影\n（15-25枚推奨）', fontsize=12, fontweight='bold')
ax.axis('off')

# 右: 推定結果
ax = axes[2]
ax.text(0.5, 0.85, '推定されるパラメータ', ha='center', fontsize=14, fontweight='bold',
        transform=ax.transAxes)

params_text = """
内部パラメータ K:
┌─────────────────┐
│ 520.5   0    320.2 │
│   0   520.1  240.1 │
│   0     0      1   │
└─────────────────┘

歪み係数:
k1 = -0.281
k2 =  0.074
p1 =  0.001
p2 = -0.002
k3 =  0.000

再投影誤差: 0.35 pixels
"""
ax.text(0.1, 0.15, params_text, fontsize=10, family='monospace',
        transform=ax.transAxes, verticalalignment='bottom')
ax.set_title('3. パラメータ推定結果', fontsize=12, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

---

## 2. キャリブレーションターゲット

### 2.1 チェッカーボード

最も一般的なキャリブレーションターゲットです。

**利点:**
- コーナー検出が高精度
- 印刷が容易
- サブピクセル精度で検出可能

**設計のポイント:**
- 非対称な格子数（例: 9×6）にすると向きが一意に決まる
- 十分なコントラスト
- 平面度が重要

In [ ]:
# ============================================================
# チェッカーボードの生成と検出シミュレーション
# ============================================================

def create_checkerboard(rows, cols, square_size=1.0):
    """
    チェッカーボードの3Dコーナー座標を生成
    
    Parameters:
    -----------
    rows : int
        内部コーナーの行数
    cols : int
        内部コーナーの列数
    square_size : float
        マス目のサイズ（任意の単位）
    
    Returns:
    --------
    corners_3d : np.ndarray (rows*cols, 3)
        コーナーの3D座標（Z=0の平面上）
    """
    corners = []
    for i in range(rows):
        for j in range(cols):
            corners.append([j * square_size, i * square_size, 0])
    return np.array(corners, dtype=np.float32)


def draw_checkerboard_image(rows, cols, square_pixels=50, margin=20):
    """
    チェッカーボード画像を生成
    """
    height = (rows + 1) * square_pixels + 2 * margin
    width = (cols + 1) * square_pixels + 2 * margin
    
    image = np.ones((height, width), dtype=np.uint8) * 255
    
    for i in range(rows + 1):
        for j in range(cols + 1):
            if (i + j) % 2 == 0:
                y_start = margin + i * square_pixels
                x_start = margin + j * square_pixels
                image[y_start:y_start+square_pixels, x_start:x_start+square_pixels] = 0
    
    # コーナー座標
    corners = []
    for i in range(rows):
        for j in range(cols):
            x = margin + (j + 1) * square_pixels
            y = margin + (i + 1) * square_pixels
            corners.append([x, y])
    
    return image, np.array(corners)


# 生成
rows, cols = 6, 9  # 非対称
checker_image, corners_2d = draw_checkerboard_image(rows, cols)
corners_3d = create_checkerboard(rows, cols, square_size=25)  # 25mm

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# チェッカーボード画像
ax = axes[0]
ax.imshow(checker_image, cmap='gray')
ax.scatter(corners_2d[:, 0], corners_2d[:, 1], c='red', s=30, marker='o')
ax.set_title(f'チェッカーボード ({cols}×{rows} 内部コーナー)', fontsize=12, fontweight='bold')
ax.axis('off')

# 3Dコーナー座標
ax = axes[1]
ax.scatter(corners_3d[:, 0], corners_3d[:, 1], c='blue', s=50)
ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_title('3Dコーナー座標（Z=0平面）', fontsize=12, fontweight='bold')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"コーナー数: {rows * cols}")
print(f"3D座標の例（最初の5点）:")
print(corners_3d[:5])

---

## 3. Zhang法（平面パターン法）

### 3.1 基本原理

Zhang法は、平面パターン（チェッカーボード）を複数の角度から撮影し、ホモグラフィから内部パラメータを推定します。

**手順:**
1. 複数の角度からチェッカーボードを撮影
2. 各画像でコーナーを検出
3. 各画像のホモグラフィを計算
4. ホモグラフィから内部パラメータの制約を導出
5. 内部パラメータを閉形式で解く
6. 非線形最適化で精緻化

### 3.2 ホモグラフィと内部パラメータ

Z=0平面上の点 $(X, Y, 0)$ と画像上の点 $(u, v)$ の関係：

$$s \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = K [r_1 | r_2 | t] \begin{bmatrix} X \\ Y \\ 1 \end{bmatrix} = H \begin{bmatrix} X \\ Y \\ 1 \end{bmatrix}$$

ここで $H = K [r_1 | r_2 | t]$ がホモグラフィ行列です。

In [ ]:
# ============================================================
# ホモグラフィの計算
# ============================================================

def compute_homography(src_points, dst_points):
    """
    DLT（Direct Linear Transform）でホモグラフィを計算
    
    src_points: (N, 2) - 元の点（3D点のX, Y）
    dst_points: (N, 2) - 画像上の点
    
    H @ [x, y, 1]^T = s * [u, v, 1]^T
    """
    N = src_points.shape[0]
    assert N >= 4, "最低4点必要"
    
    # 正規化
    src_mean = src_points.mean(axis=0)
    src_std = src_points.std()
    src_norm = (src_points - src_mean) / src_std
    T_src = np.array([
        [1/src_std, 0, -src_mean[0]/src_std],
        [0, 1/src_std, -src_mean[1]/src_std],
        [0, 0, 1]
    ])
    
    dst_mean = dst_points.mean(axis=0)
    dst_std = dst_points.std()
    dst_norm = (dst_points - dst_mean) / dst_std
    T_dst = np.array([
        [1/dst_std, 0, -dst_mean[0]/dst_std],
        [0, 1/dst_std, -dst_mean[1]/dst_std],
        [0, 0, 1]
    ])
    
    # DLT行列を構築
    A = []
    for i in range(N):
        x, y = src_norm[i]
        u, v = dst_norm[i]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    
    A = np.array(A)
    
    # SVDで解く
    _, _, Vt = np.linalg.svd(A)
    H_norm = Vt[-1].reshape(3, 3)
    
    # 正規化を戻す
    H = np.linalg.inv(T_dst) @ H_norm @ T_src
    H = H / H[2, 2]  # 正規化
    
    return H


# シミュレーション: 仮想カメラでチェッカーボードを撮影
def simulate_camera_view(corners_3d, K, R, t, noise_std=0.5):
    """
    仮想カメラでの撮影をシミュレート
    """
    # カメラ座標に変換
    corners_cam = (R @ corners_3d.T).T + t
    
    # 射影
    corners_norm = corners_cam[:, :2] / corners_cam[:, 2:3]
    corners_2d = (K[:2, :2] @ corners_norm.T).T + K[:2, 2]
    
    # ノイズ追加
    if noise_std > 0:
        corners_2d += np.random.randn(*corners_2d.shape) * noise_std
    
    return corners_2d


# 真のカメラパラメータ
K_true = np.array([
    [520, 0, 320],
    [0, 520, 240],
    [0, 0, 1]
], dtype=np.float64)

# 複数の視点でシミュレーション
def rotation_matrix_z(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def rotation_matrix_x(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])

# 視点ごとの外部パラメータ
views = [
    (rotation_matrix_x(np.radians(30)), np.array([50, 30, 500])),
    (rotation_matrix_x(np.radians(30)) @ rotation_matrix_z(np.radians(20)), np.array([70, 40, 480])),
    (rotation_matrix_x(np.radians(25)) @ rotation_matrix_z(np.radians(-15)), np.array([30, 50, 520])),
]

# 各視点での2D点を生成
all_corners_2d = []
for R, t in views:
    corners_2d = simulate_camera_view(corners_3d, K_true, R, t, noise_std=0.3)
    all_corners_2d.append(corners_2d)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (corners_2d, ax) in enumerate(zip(all_corners_2d, axes)):
    ax.scatter(corners_2d[:, 0], corners_2d[:, 1], c='red', s=20)
    ax.set_xlim(0, 640)
    ax.set_ylim(480, 0)
    ax.set_aspect('equal')
    ax.set_title(f'視点 {i+1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('u (pixels)')
    ax.set_ylabel('v (pixels)')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Zhang法による内部パラメータ推定（簡略版）
# ============================================================

def zhang_calibration(object_points, image_points_list):
    """
    Zhang法によるカメラキャリブレーション（簡略版）
    
    Parameters:
    -----------
    object_points : np.ndarray (N, 3)
        3Dオブジェクト点（Z=0）
    image_points_list : list of np.ndarray (N, 2)
        各画像での2D点
    
    Returns:
    --------
    K : np.ndarray (3, 3)
        推定された内部パラメータ行列
    """
    # Step 1: 各画像のホモグラフィを計算
    homographies = []
    for image_points in image_points_list:
        H = compute_homography(object_points[:, :2], image_points)
        homographies.append(H)
    
    # Step 2: V行列を構築（Zhangの制約条件）
    def vij(H, i, j):
        """Zhangの論文のvij関数"""
        return np.array([
            H[0, i] * H[0, j],
            H[0, i] * H[1, j] + H[1, i] * H[0, j],
            H[1, i] * H[1, j],
            H[2, i] * H[0, j] + H[0, i] * H[2, j],
            H[2, i] * H[1, j] + H[1, i] * H[2, j],
            H[2, i] * H[2, j]
        ])
    
    V = []
    for H in homographies:
        V.append(vij(H, 0, 1))
        V.append(vij(H, 0, 0) - vij(H, 1, 1))
    
    V = np.array(V)
    
    # Step 3: SVDでBを解く
    _, _, Vt = np.linalg.svd(V)
    b = Vt[-1]
    
    # B = K^{-T} K^{-1} の要素
    B11, B12, B22, B13, B23, B33 = b
    
    # Step 4: Bから内部パラメータを抽出
    v0 = (B12 * B13 - B11 * B23) / (B11 * B22 - B12 * B12)
    lambda_ = B33 - (B13 * B13 + v0 * (B12 * B13 - B11 * B23)) / B11
    alpha = np.sqrt(lambda_ / B11)
    beta = np.sqrt(lambda_ * B11 / (B11 * B22 - B12 * B12))
    gamma = -B12 * alpha * alpha * beta / lambda_
    u0 = gamma * v0 / beta - B13 * alpha * alpha / lambda_
    
    K = np.array([
        [alpha, gamma, u0],
        [0, beta, v0],
        [0, 0, 1]
    ])
    
    return K, homographies


# キャリブレーション実行
K_estimated, homographies = zhang_calibration(corners_3d, all_corners_2d)

print("真の内部パラメータ K:")
print(np.round(K_true, 2))
print()
print("推定された内部パラメータ K:")
print(np.round(K_estimated, 2))
print()
print(f"焦点距離の誤差: fx={abs(K_true[0,0]-K_estimated[0,0]):.2f}, fy={abs(K_true[1,1]-K_estimated[1,1]):.2f}")
print(f"主点の誤差: cx={abs(K_true[0,2]-K_estimated[0,2]):.2f}, cy={abs(K_true[1,2]-K_estimated[1,2]):.2f}")

---

## 4. 再投影誤差

### 4.1 再投影誤差の定義

**再投影誤差（Reprojection Error）**は、キャリブレーション精度の指標です：

$$E = \frac{1}{NM} \sum_{i=1}^{M} \sum_{j=1}^{N} ||\mathbf{x}_{ij} - \hat{\mathbf{x}}_{ij}||^2$$

- $\mathbf{x}_{ij}$: 検出された2D点
- $\hat{\mathbf{x}}_{ij}$: 推定パラメータで再投影した点
- $M$: 画像数
- $N$: 点数

In [ ]:
# ============================================================
# 再投影誤差の計算と可視化
# ============================================================

def compute_reprojection_error(object_points, image_points, K, H):
    """
    再投影誤差を計算
    """
    # ホモグラフィで投影
    pts_h = np.hstack([object_points[:, :2], np.ones((len(object_points), 1))])
    projected = (H @ pts_h.T).T
    projected = projected[:, :2] / projected[:, 2:3]
    
    # 誤差
    errors = np.linalg.norm(image_points - projected, axis=1)
    return errors, projected


# 全視点での再投影誤差
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

total_errors = []

for i, (image_points, H, ax) in enumerate(zip(all_corners_2d, homographies, axes)):
    errors, projected = compute_reprojection_error(corners_3d, image_points, K_estimated, H)
    total_errors.extend(errors)
    
    # 元の点
    ax.scatter(image_points[:, 0], image_points[:, 1], c='blue', s=30, label='検出点', alpha=0.7)
    # 再投影点
    ax.scatter(projected[:, 0], projected[:, 1], c='red', s=30, marker='x', label='再投影点')
    
    # 誤差ベクトル
    for j in range(len(image_points)):
        ax.plot([image_points[j, 0], projected[j, 0]], 
                [image_points[j, 1], projected[j, 1]], 'g-', alpha=0.5, linewidth=0.5)
    
    ax.set_xlim(0, 640)
    ax.set_ylim(480, 0)
    ax.set_aspect('equal')
    ax.set_title(f'視点 {i+1}\n平均誤差: {np.mean(errors):.2f} px', fontsize=11, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 誤差の統計
total_errors = np.array(total_errors)
print(f"再投影誤差の統計:")
print(f"  平均: {np.mean(total_errors):.3f} pixels")
print(f"  標準偏差: {np.std(total_errors):.3f} pixels")
print(f"  最大: {np.max(total_errors):.3f} pixels")
print(f"  RMS: {np.sqrt(np.mean(total_errors**2)):.3f} pixels")

---

## 5. 非線形最適化による精緻化

### 5.1 バンドル調整

初期推定の後、以下の目的関数を最小化して精緻化します：

$$\min_{K, k_i, R_j, t_j} \sum_{i,j} ||\mathbf{x}_{ij} - \pi(K, k, R_j, t_j, \mathbf{X}_i)||^2$$

- $K$: 内部パラメータ
- $k$: 歪み係数
- $R_j, t_j$: 各画像のカメラポーズ
- $\mathbf{X}_i$: 3D点
- $\pi$: 射影関数（歪みを含む）

In [ ]:
# ============================================================
# 非線形最適化による精緻化（簡略版）
# ============================================================

from scipy.optimize import least_squares

def project_points_with_distortion(object_points, K, R, t, dist_coeffs):
    """
    歪みを含む射影
    """
    # カメラ座標に変換
    pts_cam = (R @ object_points.T).T + t
    
    # 正規化座標
    x = pts_cam[:, 0] / pts_cam[:, 2]
    y = pts_cam[:, 1] / pts_cam[:, 2]
    
    # 放射歪み
    r2 = x**2 + y**2
    k1, k2, p1, p2, k3 = dist_coeffs
    radial = 1 + k1 * r2 + k2 * r2**2 + k3 * r2**3
    
    # 接線歪み
    x_d = x * radial + 2 * p1 * x * y + p2 * (r2 + 2 * x**2)
    y_d = y * radial + p1 * (r2 + 2 * y**2) + 2 * p2 * x * y
    
    # ピクセル座標
    u = K[0, 0] * x_d + K[0, 2]
    v = K[1, 1] * y_d + K[1, 2]
    
    return np.stack([u, v], axis=1)


def calibration_residuals(params, object_points, image_points_list, n_images):
    """
    最適化の残差関数
    """
    # パラメータを展開
    fx, fy, cx, cy = params[:4]
    dist_coeffs = params[4:9]
    
    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ])
    
    residuals = []
    
    for i in range(n_images):
        # 各画像の外部パラメータ
        offset = 9 + i * 6
        rvec = params[offset:offset+3]
        tvec = params[offset+3:offset+6]
        
        # 回転ベクトルから回転行列へ
        theta = np.linalg.norm(rvec)
        if theta > 1e-10:
            k = rvec / theta
            K_skew = np.array([[0, -k[2], k[1]], [k[2], 0, -k[0]], [-k[1], k[0], 0]])
            R = np.eye(3) + np.sin(theta) * K_skew + (1 - np.cos(theta)) * (K_skew @ K_skew)
        else:
            R = np.eye(3)
        
        # 射影
        projected = project_points_with_distortion(object_points, K, R, tvec, dist_coeffs)
        
        # 残差
        res = (image_points_list[i] - projected).flatten()
        residuals.extend(res)
    
    return np.array(residuals)


print("💡 実際のキャリブレーションではOpenCVのcv2.calibrateCamera()を使用します")
print("   これは非線形最適化（Levenberg-Marquardt法）を内部で実行します")

---

## 6. 実践的な考慮事項

### 6.1 撮影のガイドライン

| 項目 | 推奨 |
|------|------|
| 画像数 | 15-25枚 |
| 角度 | さまざまな角度（傾き±45度程度まで）|
| 位置 | 画像全体をカバー |
| 照明 | 均一、反射を避ける |
| フォーカス | シャープに（ボケNG）|

### 6.2 精度の目安

| 再投影誤差 | 評価 |
|------------|------|
| < 0.5 pixels | 非常に良い |
| 0.5-1.0 pixels | 良い |
| 1.0-2.0 pixels | 普通 |
| > 2.0 pixels | 要改善 |

In [ ]:
# ============================================================
# キャリブレーション品質の評価
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 再投影誤差のヒストグラム
ax = axes[0]
ax.hist(total_errors, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(np.mean(total_errors), color='red', linestyle='--', linewidth=2, label=f'平均: {np.mean(total_errors):.2f} px')
ax.axvline(np.median(total_errors), color='green', linestyle='--', linewidth=2, label=f'中央値: {np.median(total_errors):.2f} px')
ax.set_xlabel('再投影誤差 (pixels)', fontsize=12)
ax.set_ylabel('頻度', fontsize=12)
ax.set_title('再投影誤差の分布', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 右: 誤差の方向性（バイアスチェック）
ax = axes[1]

all_errors_xy = []
for image_points, H in zip(all_corners_2d, homographies):
    pts_h = np.hstack([corners_3d[:, :2], np.ones((len(corners_3d), 1))])
    projected = (H @ pts_h.T).T
    projected = projected[:, :2] / projected[:, 2:3]
    errors_xy = image_points - projected
    all_errors_xy.append(errors_xy)

all_errors_xy = np.vstack(all_errors_xy)

ax.scatter(all_errors_xy[:, 0], all_errors_xy[:, 1], alpha=0.5, s=20)
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(0, color='gray', linestyle='--', alpha=0.5)
ax.scatter([0], [0], c='red', s=100, marker='+', linewidths=3, label='原点')

# 平均誤差
mean_err = all_errors_xy.mean(axis=0)
ax.scatter([mean_err[0]], [mean_err[1]], c='green', s=100, marker='x', linewidths=3, label=f'平均 ({mean_err[0]:.2f}, {mean_err[1]:.2f})')

ax.set_xlabel('X方向の誤差 (pixels)', fontsize=12)
ax.set_ylabel('Y方向の誤差 (pixels)', fontsize=12)
ax.set_title('誤差の方向分布\n（バイアスチェック）', fontsize=12, fontweight='bold')
ax.legend()
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 誤差の分布が原点周りに対称なら、系統的なバイアスはありません")

---

## まとめ

### 🎯 このノートブックで学んだこと

**キャリブレーションの目的**
- ✓ 内部パラメータ（K）の推定
- ✓ 歪み係数の推定
- ✓ 外部パラメータの推定（各画像）

**Zhang法**
- ✓ 平面パターンを複数角度から撮影
- ✓ ホモグラフィから制約を導出
- ✓ 閉形式解 + 非線形最適化

**精度評価**
- ✓ 再投影誤差（RMS）
- ✓ 誤差の分布分析

### 📊 キャリブレーションチェックリスト

| ステップ | 確認事項 |
|----------|----------|
| 準備 | チェッカーボードが平面で歪みがないか |
| 撮影 | 15-25枚、さまざまな角度、画像全体をカバー |
| 検出 | すべてのコーナーが正しく検出されているか |
| 結果 | 再投影誤差 < 1.0 pixels |
| 検証 | 歪み補正後の直線が真っ直ぐか |

---

## 🎓 自己評価クイズ

### Q1: なぜチェッカーボードは非対称な格子数（例: 9×6）が推奨されますか？

<details>
<summary>💡 答えを見る</summary>

非対称にすることで、チェッカーボードの向き（上下左右）が一意に決まります。対称な格子（8×8など）では、180度回転しても同じに見えるため、向きの曖昧さが生じます。

</details>

---

### Q2: 再投影誤差とは何ですか？

<details>
<summary>💡 答えを見る</summary>

**再投影誤差**は、推定されたカメラパラメータを使って3D点を画像に投影した点と、実際に検出された2D点との距離です。

$$E = ||\mathbf{x}_{detected} - \mathbf{x}_{projected}||$$

キャリブレーション精度の指標として使用され、一般的に1ピクセル以下が良いとされます。

</details>

---

### Q3: キャリブレーション画像は何枚必要ですか？

<details>
<summary>💡 答えを見る</summary>

理論的には最低3枚ですが、実用的には**15-25枚**が推奨されます。

理由：
- ノイズの影響を平均化できる
- さまざまな角度をカバーできる
- パラメータの信頼区間を狭くできる

</details>

---

### ✅ 学習チェックリスト

- [ ] キャリブレーションで推定するパラメータを列挙できる
- [ ] Zhang法の手順を説明できる
- [ ] 再投影誤差を計算できる
- [ ] キャリブレーション品質を評価できる

---

**次のステップ**: Notebook 55で、**エピポーラ幾何**を学びます！

2視点間の幾何学的関係を理解し、基礎行列・本質行列を学びます。